In [5]:
from agents import *
from random import randrange, seed, choice

# A Mining * is Born!
This program simulates mining for gold. Unlike the previous assignment, this time the robot can actually sense what is around it! We will set the perceptible distance to 2, giving the robot knowledge of a small slice of its grid world. One thing to keep in mind is that everything the robot sees will be a Thing, and things have "location" variables which indicate their exact coordinate.

In order to help our little buddy out, we are going to code an A* search algorithm to get it onto desired squares.

## The Environment
First, we define a Gold class as well as a graphical environment for the robot to explore, we then generate a random mine. Read over this code, but there is nothing that needs changing here. The environment is drawn as a grid. Black squares are walls, which the robot cannot pass through. The gold squares are squares containing gold and the grey squares are empty squares. Finally, when we add the robot, it will show up as a red square. Perciptible distance is set to 2.

In [6]:
class Gold(Thing):
    """
    A chunk of gold, it does nothing! (But the Wumpus gold in agents is a singleton)
    so we need this one here.
    """
    pass

class MineEnvironment(GraphicEnvironment):
    def __init__(self):
        color = {"Miner": (255,0,0),
                 "Gold": (253, 208, 23),
                 "Wall": (44, 53, 57)}
        super().__init__(16, 16, True, color)
        self.perceptible_distance=2
        self.add_walls()

        # place obstacles
        for i in range(64):
            self.add_thing(Wall(), self.empty_square())
        
        # place gold
        for i in range(42):
            self.add_thing(Gold(), self.empty_square())
    
    def empty_square(self):
        world = self.get_world()
        done = False
        while not done:
            x = randrange(len(world))
            y = randrange(len(world[x]))
            done = len(world[x][y]) == 0
        return x,y

mine = MineEnvironment()
mine.reveal()

,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,


## A Star Miner
Now we come to the star of the show! Our intrepid little miner robot. The robot enters the field on a random empty square. Its constructor sets up the robots program and sets its initial direction to facing to the right. The robot can perform the following operations:

- TurnLeft
- TurnRight
- Forward
- Grab

These actions are perfomed by returning them from the program function. The program function receives an array of percepts, which is an array of tuples consisting of the objects on the robot's current square, as well as its **squared straight line distance** from the robot. A sample array is shown below.

```
[(<Gold>, 4), (<Wall>, 1), (<Wall>, 2), (<Miner>, 0)]
```

The above array would mean that there is a gold brick 2 units away, a wall 1 unit away, a wall sqrt(2) units away and miner 0 units away (the miner will always be there.).

In addition to the percept sequence, there is also the variable `self.bump` which is true if the robot rolled into a wall on its last move.

Note how the Miner is always included in the percept list. This is because the miner is part of the environment too! If you attempt to "Grab", the environment makes sure this is something that can be grabbed by calling the "can_grab" function.

## Your Task
Using the supplied miner code, modify the program to use an A* based search to catch as much gold as you can. Be sure to add functions to compute f(n) = g(n) + h(n). 

**Hints**
- You can slowly discover the entire mine if you keep a clever variable.
- Select and update goals based on the nearest and greatest gold vein. This can
  be independent of the A* computation.
- A* only finds the most effecient square which brings you closer to the goal square. So the task here is 
  to select a good goal and then use A* to get there.
- You can see the walls now! Don't target wall squares. Treat them as unreachable nodes
  when doing A* (they have infinite cost).
- Keep it simple. You don't have to write much code to get an effective solution.
- Remember that the total cost to a square depends on which way you are facing.
  be sure to pay attention to that direction variable when computing your f(n) values.

In [7]:
class Miner(Agent):
    def __init__(self, program=None):
        super().__init__(self.program)
        self.direction = Direction("right")
        self.history=[]
        self.plan=[]
        
    def can_grab(self, thing):
        return thing.__class__.__name__ == "Gold"
    
    def program(self, percept):
        #TODO: Do something better rolling in our Konami path
        if len(self.plan) == 0:
            self.plan = [self.up, self.up, self.down, self.down, self.left, self.right, self.left, self.right]
        
        # execute the next part of the plan
        return self.plan[0]()
   
    def face_advance(self, d1, d2):
        """
        Attempt to roll in direction d1. Pops the plan if it succeeds.
        Otherwise, if facing in d2, turn right, otherwise turn left.
        """
        if self.direction.direction == d1:
            self.plan.pop(0)
            return "Forward"
        elif self.direction.direction == d2:
            return "TurnRight"
        else:
            return "TurnLeft"

    def up(self):
        """
        Execute the next step in a plan to go up, popping when down.
        """
        return self.face_advance(Direction.U, Direction.L)
    
    def down(self):
        """
        Execute the next step in a plan to go down, popping when down.
        """
        return self.face_advance(Direction.D, Direction.R)
    
    def left(self):
        """
        Execute the next step in a plan to go left, popping when down.
        """
        return self.face_advance(Direction.L, Direction.D)
    
    def right(self):
        """
        Execute the next step in a plan to go right, popping when down.
        """
        return self.face_advance(Direction.R, Direction.U)
        

miner = Miner()
mine.add_thing(miner, mine.empty_square())
mine.run(50)

,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,


## How did we do?
The code below displays the robot's final amount of collected gold.

In [8]:
print("Gold Collected: {}".format(len(miner.holding)))


Gold Collected: 0
